In [13]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from typing import List, Dict

In [18]:
with open('client_secret.txt', 'r') as file:
    client_secret_key = file.read()
    file.close()

def setup_spotify_client() -> spotipy.Spotify:
    """
    Set up and return an authenticated Spotify client
    """
    # You'll need to set these up in your Spotify Developer Dashboard
    CLIENT_ID = '5394f0b848b5418cbaf20409aae2eb05'
    CLIENT_SECRET = client_secret_key
    REDIRECT_URI = 'http://localhost:8888/callback'
    SCOPE = 'user-library-read'
    
    # Create the SpotifyOAuth object
    auth_manager = SpotifyOAuth(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        redirect_uri=REDIRECT_URI,
        scope=SCOPE
    )
    
    # Get access token (this will prompt auth if needed)
    token_info = auth_manager.get_cached_token()
    if not token_info:
        # Get new token
        token_info = auth_manager.get_access_token()
    
    # Check if token needs refresh
    if auth_manager.is_token_expired(token_info):
        token_info = auth_manager.refresh_access_token(token_info['refresh_token'])
    
    # Return authenticated client
    return spotipy.Spotify(auth_manager=auth_manager)

def get_all_liked_songs(sp: spotipy.Spotify) -> List[Dict]:
    """
    Retrieve all liked songs from the user's library
    """
    songs = []
    offset = 0
    limit = 50  # Maximum allowed by Spotify API
    
    while True:
        results = sp.current_user_saved_tracks(limit=limit, offset=offset)
        if not results['items']:
            break
            
        for item in results['items']:
            track = item['track']
            
            # Extract artist names
            artists = ', '.join([artist['name'] for artist in track['artists']])
            
            # Extract relevant track information
            # Extract only requested parameters
            song_info = {
                'name': track['name'],
                'artists': artists,
                'album': track['album']['name'],
                'release_date': track['album']['release_date'],
                'duration_ms': track['duration_ms'],
                'popularity': track['popularity'],
                'explicit': track['explicit'],
                'added_at': item['added_at']
            }
            
            # If you want audio features (requires additional API call)
            try:
                audio_features = sp.audio_features(track['id'])[0]
                if audio_features:
                    song_info.update({
                        'danceability': audio_features['danceability'],
                        'energy': audio_features['energy'],
                        'key': audio_features['key'],
                        'loudness': audio_features['loudness'],
                        'mode': audio_features['mode'],
                        'speechiness': audio_features['speechiness'],
                        'acousticness': audio_features['acousticness'],
                        'instrumentalness': audio_features['instrumentalness'],
                        'liveness': audio_features['liveness'],
                        'valence': audio_features['valence'],
                        'tempo': audio_features['tempo'],
                        'time_signature': audio_features['time_signature']
                    })
            except:
                pass
            songs.append(song_info)
            
        offset += limit
    
    return songs

# GET TOKEN TO ACCESS SONG PARAMETER DATA

In [ ]:
sp = setup_spotify_client()

print("Fetching liked songs...")
songs = get_all_liked_songs(sp)

df = pd.DataFrame(songs)

df['duration_min'] = df['duration_ms'] / 60000

# df['release_date'] = pd.to_datetime(df['release_date'])
# df['added_at'] = pd.to_datetime(df['added_at'])

df = df.sort_values('added_at', ascending=False)


# Display basic information
print(f"\nTotal number of liked songs: {len(df)}")
print("\nMost recent songs added:")
print(df[['name', 'artists', 'added_at']].head())

Fetching liked songs...


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4x94qgTeMEcwj4X1APUnpb with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2Rr4ZOnKXUrzjhMPkmkRjK with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1FNvrp5K02oThumS6gllzp with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0rAdstq2FtKpApOYyouwAf with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1FMx9xm0tf53HCh1T6TXK0 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=16IzV3JdxmR3X10ZbWHlfB with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3aGLLhUSjtJuCz0V6tg4pn with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=


Total number of liked songs: 174

Most recent songs added:
                  name                artists              added_at
0                 Girl               Jamie xx  2024-12-11T14:57:08Z
1         Soup for One                   CHIC  2024-12-11T14:42:27Z
2  Block Rockin' Beats  The Chemical Brothers  2024-12-11T14:38:25Z
3   Spatial Perception     Enrico Sangiuliano  2024-11-27T07:00:01Z
4              No Good             Joe Mattei  2024-11-23T22:43:17Z


In [17]:
df.columns

Index(['name', 'artists', 'album', 'release_date', 'duration_ms', 'popularity',
       'explicit', 'added_at', 'duration_min'],
      dtype='object')